In [10]:
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_and_add_tool_responses
from agentic_blocks.utils.tools_utils import execute_pending_tool_calls
from agentic_blocks.utils.tools_utils import langchain_tool_to_openai_format
from agentic_blocks import call_llm, Messages
from deep_research.tools import internet_search, write_todos, think_tool
from IPython.display import display, Markdown

In [4]:
tools = [internet_search, write_todos, think_tool]

tool_registry = create_tool_registry(tools)

#langchain_tool_to_openai_format(think_tool)

In [5]:
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

First use the write_todos tool to create a todo list.

When you have a list of todos, use the internet_search tool to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report provide the final report.

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!

Note: Write the report in English.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in English.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to these tools:

## `write_todos`

Use this to create and manage a structured task list. Pass an array of todo objects, each with:
- `content`: A string describing the task (required)
- `status`: One of "pending", "in_progress", or "completed" (optional, defaults to "pending")

Example: write_todos(todos=[{"content": "Research top restaurants", "status": "pending"}, {"content": "Write report", "status": "pending"}])

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.
"""

In [12]:
messages = Messages(
    system_prompt=research_instructions,
    user_prompt="Make a thorough comparison of Eight Sleep and Bedjet wit pros and cons"
)

model = "qwen/qwen3-235b-a22b-2507"
#model = "openai/gpt-oss-120b"

In [13]:
has_answer = False

while not has_answer:
    response = call_llm(model=model, messages=messages, tools=tools)
    messages.add_response_message(response)
    #print(messages.get_messages()[-1])
    
    if messages.has_pending_tool_calls():
        for tool_call in messages.get_pending_tool_calls():
            print(f"## Tool call: {tool_call['tool_name']}: {tool_call['arguments']}")
        #print(messages.get_pending_tool_calls())
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
    else:
        has_answer = True
    
    

## Tool call: write_todos: {'todos': [{'content': 'Research Eight Sleep and Bedjet products, focusing on features, technology, and user experience', 'status': 'pending'}, {'content': 'Compare pros and cons of Eight Sleep vs Bedjet for different user needs', 'status': 'pending'}, {'content': 'Summarize key differences and provide recommendations based on sleep preferences', 'status': 'pending'}]}
-------------- Writing todos --------------
- [pending] Research Eight Sleep and Bedjet products, focusing on features, technology, and user experience
- [pending] Compare pros and cons of Eight Sleep vs Bedjet for different user needs
- [pending] Summarize key differences and provide recommendations based on sleep preferences
## Tool call: internet_search: {'query': 'Eight Sleep pod pro cover features technology'}
## Tool call: think_tool: {'reflection': "I have gathered initial information about Eight Sleep products, particularly the Pod series which features temperature regulation, sleep tra

In [14]:
display(Markdown(response.content))

# Eight Sleep vs Bedjet: Comprehensive Comparison of Bed Cooling Systems

## Overview of Eight Sleep and Bedjet Technologies

Eight Sleep and Bedjet represent two distinct approaches to bed climate control, with fundamentally different technologies and feature sets. Eight Sleep utilizes a hydro-powered system that integrates into a mattress cover, using water-based temperature regulation to cool or heat the bed through embedded tubes. The system tracks biometrics like heart rate, respiration, and sleep stages, offering comprehensive sleep data and health insights [Eight Sleep Pod 3 Review](https://www.popularmechanics.com/technology/a60165942/eight-sleep-review/). In contrast, Bedjet employs an air-based forced air system that pumps conditioned air directly into the bedding through a nozzle positioned at the foot of the bed. The Bedjet 3 uses a base unit that sits beside or under the bed and can either blow air directly over the body or through a specialized "Cloudsheet" for more even distribution [BedJet 3 Review](https://www.youtube.com/watch?v=t6i1j3tw8mQ).

The core technological difference—water-based vs. air-based cooling—fundamentally shapes each product's performance, installation requirements, and user experience. Eight Sleep's water circulation system provides more precise temperature control and can achieve lower temperatures than Bedjet's air system, but requires more complex installation and maintenance. Bedjet's air-based approach offers rapid temperature changes and a simpler setup process, but may be more noticeable to users due to airflow across the skin. Eight Sleep positions itself as a comprehensive sleep optimization platform with health tracking capabilities, while Bedjet focuses primarily on thermal comfort with some sleep enhancement features like programmable temperature cycles and wake-up functions.

## Eight Sleep: Pros and Cons

### Advantages of Eight Sleep

Eight Sleep offers several compelling advantages for users seeking comprehensive sleep optimization. The hydro-powered temperature regulation provides precise control, capable of cooling as low as 55°F (13°C) and heating up to 110°F (43°C), offering a wider temperature range than air-based systems [Eight Sleep Pod 4 Cover](https://www.cnet.com/health/sleep/we-tested-the-eight-sleep-pod-4-cover/). The system features true dual-zone temperature control, allowing partners to set completely different temperatures on each side of the bed without thermal crossover, which is particularly beneficial for couples with differing temperature preferences. Eight Sleep integrates extensive sleep tracking capabilities, monitoring heart rate, respiration rate, sleep stages, and nocturnal movements through sensors in the mattress cover, providing detailed sleep performance data and personalized insights.

Additional benefits include the GentleRise wake-up technology that gradually raises the bed temperature to simulate sunrise and wake users more naturally, smart home integration with platforms like Apple Health and Google Fit, and app-controlled automation that learns user preferences over time. The system works with most standard mattresses as it's a cover that wraps around the existing mattress, preserving previous mattress investments. For users committed to optimizing their sleep health, Eight Sleep provides a more holistic solution that combines thermal comfort with comprehensive biometric tracking and analysis.

### Disadvantages of Eight Sleep

Despite its advanced features, Eight Sleep has several notable drawbacks. The system has a significantly higher price point, with the Pod 4 Cover starting at around $2,395 for a Queen size, making it considerably more expensive than air-based alternatives [BedJet Vs Eight Sleep](https://www.skool.com/review-7926/bedjet-vs-eight-sleep). Installation is more complex, requiring users to fill the hub with distilled water, prime the system, and connect to WiFi, with occasional maintenance needed to prevent algae growth in the water tubes. The water-based system requires a larger footprint with the control hub that sits beside the bed, taking up more space than compact air units.

Some users report reliability concerns with the water circulation system, including potential leaks (though rare) and the need for regular maintenance. The app and system interface has received mixed reviews, with some users finding it overly complex or experiencing connectivity issues. Additionally, the mattress cover design means users must commit to sleeping on the Eight Sleep system exclusively, as switching back to a traditional bed setup requires removing the cover. The noise level, while generally quiet, includes a faint hum from the water pump that some light sleepers may notice.

## Bedjet: Pros and Cons

### Advantages of Bedjet

Bedjet offers several significant advantages that make it appealing to many consumers. The most notable benefit is its considerably lower price point, with the Bedjet 3 system costing less than $500, making it accessible to a broader range of consumers compared to Eight Sleep's premium pricing [BedJet Vs Eight Sleep](https://www.skool.com/review-7926/bedjet-vs-eight-sleep). Installation is remarkably simple—users place the base unit beside or under the bed, insert the air nozzle between the mattress and fitted sheet at the foot of the bed, and plug it in, with setup typically taking less than 10 minutes. The air-based system delivers rapid temperature changes, with users feeling the cooling or warming effect almost immediately after activation.

The compact design of the base unit (approximately 12" x 12" x 5") takes up minimal space and can be easily stored or moved. Bedjet operates more quietly than many expect, with a "silky-quiet" fan system that produces minimal noise during operation. The system offers excellent versatility, functioning as both a cooling and warming solution, with temperature control ranging from 45°F to 106°F (7°C to 41°C) depending on ambient room conditions. For users who prefer airflow or have issues with feeling clammy, the direct air circulation can provide a refreshing sensation that some find more comfortable than water-based systems. The dual-zone option allows couples to have completely independent climate control on each side of the bed when using two units.

### Disadvantages of Bedjet

Bedjet's air-based technology has some limitations compared to water-based systems. The cooling effectiveness is dependent on ambient room temperature, meaning it cannot cool below room temperature but rather provides cool airflow relative to the current environment. In very hot environments, the perceived cooling effect may be less pronounced than with Eight Sleep's ability to actively chill water. Some users report that the airflow can feel drafty or disruptive to sleep, particularly at higher fan speeds, though this can be mitigated with the optional Cloudsheet accessory that diffuses the airflow.

The system does not offer comprehensive sleep tracking or biometric monitoring, focusing exclusively on thermal comfort rather than holistic sleep health analysis. The air nozzle at the foot of the bed can create a barrier that makes tucking in sheets more difficult, though this is a minor inconvenience for most users. While the system is generally reliable, the forced air mechanism has moving parts that could potentially fail over extended periods of use. The Bedjet 3 lacks some of the smart home integration features found in Eight Sleep, though it does offer programmable temperature settings through its biorhythm sleep technology that allows users to set different temperatures for different parts of the night.

## Key Differences and User Considerations

### Technology and Performance Comparison

The fundamental difference between Eight Sleep and Bedjet lies in their core technologies: Eight Sleep uses a water-based hydro system that circulates temperature-controlled water through tubes in a mattress cover, while Bedjet employs an air-based system that blows conditioned air directly into the bedding. This technological distinction creates different performance characteristics—Eight Sleep provides more precise temperature control and can actively chill below ambient room temperature, while Bedjet offers rapid airflow changes but cannot cool below the room's baseline temperature [BedJet vs Eight Sleep](https://www.rd.com/article/bedjet-vs-eight-sleep/).

In terms of cooling effectiveness, Eight Sleep generally provides a more consistent and profound cooling experience, particularly beneficial for hot sleepers or those in very warm climates. The water-based system maintains temperature more steadily throughout the night, while Bedjet's air system may require periodic cycling to maintain the desired comfort level. For heating, both systems perform well, though Eight Sleep's radiant heat from the mattress may feel more natural and comfortable than Bedjet's airflow-based warming. The responsiveness differs significantly—Bedjet delivers almost immediate temperature changes (within 30 seconds), while Eight Sleep takes several minutes to reach the target temperature due to water circulation requirements.

### Price, Installation, and Maintenance

Price is one of the most significant differentiators between the two systems. Eight Sleep represents a premium investment, with prices starting around $2,395 for the Pod 4 Cover, while Bedjet 3 costs approximately $499 for a single unit, making it roughly one-fifth the price of Eight Sleep [BedJet Vs Eight Sleep](https://www.skool.com/review-7926/bedjet-vs-eight-sleep). For couples wanting dual-zone climate control, Eight Sleep requires purchasing a single integrated system priced accordingly, while Bedjet allows purchasing units separately, enabling incremental investment.

Installation complexity favors Bedjet significantly. The air-based system requires minimal setup—positioning the unit, inserting the nozzle, and plugging it in—while Eight Sleep requires filling the hub with distilled water, priming the tubing system, and ensuring proper connectivity. Maintenance requirements also differ: Eight Sleep recommends periodic cleaning and descaling of the water system every 6-12 months to prevent mineral buildup and potential algae growth, while Bedjet requires minimal maintenance beyond occasional cleaning of the air filter and exterior.

### Feature Set and Sleep Optimization

Eight Sleep offers a more comprehensive suite of sleep optimization features beyond temperature control. The system tracks multiple biometrics including heart rate variability, respiration rate, sleep stages (light, deep, REM), and bed exit patterns, providing detailed sleep performance scores and personalized recommendations. Integration with health platforms like Apple Health and Fitbit allows for holistic health tracking, and the app includes guided breathing exercises and sleep coaching features. The GentleRise wake-up technology gradually increases bed temperature to facilitate natural waking, while smart scheduling can automatically adjust temperatures based on circadian rhythms.

Bedjet's feature set is more focused on thermal comfort. Its biorhythm sleep technology allows users to program temperature changes throughout the night (e.g., cooling during initial sleep, warming before wake-up), and the system includes timed operation and remote control capabilities. However, it lacks biometric tracking and health monitoring features entirely. For users primarily concerned with temperature regulation without the need for comprehensive sleep data, Bedjet's focused approach may be preferable. The simplicity of operation—large physical buttons on the unit itself—makes it accessible for users who prefer minimal app dependence or have difficulty with complex technology interfaces.

## Recommendations and Ideal Use Cases

### Best Scenarios for Eight Sleep

Eight Sleep is ideally suited for users seeking a comprehensive sleep optimization platform who are willing to invest in premium technology. It's particularly beneficial for couples with significantly different temperature preferences, as the true dual-zone control allows each partner to maintain their ideal sleep climate without compromise. The system is excellent for hot sleepers, menopausal women experiencing night sweats, or anyone in hot climates who needs active cooling below ambient temperatures. Health-conscious individuals who value biometric tracking and data-driven insights into their sleep patterns will find the extensive monitoring capabilities valuable.

Eight Sleep is also appropriate for users already invested in smart home ecosystems who appreciate integration with platforms like Apple Health, Google Fit, and smart home assistants. Those who prefer a more subtle temperature regulation experience without noticeable airflow may prefer Eight Sleep's radiant heating and cooling. The system works well for individuals seeking to optimize athletic recovery, as the temperature control and sleep data can help maximize rest and regeneration.

### Best Scenarios for Bedjet

Bedjet is ideal for budget-conscious consumers who want effective temperature regulation without the premium price tag. It's particularly suitable for users who appreciate rapid temperature changes and the sensation of airflow during sleep. The simple setup and operation make it accessible for technology-averse users or those who prefer minimal app interaction. Individuals in moderate climates who need supplemental cooling during warmer months or additional warmth during colder nights will find Bedjet's capabilities sufficient for their needs.

The system is excellent for couples where only one partner has temperature regulation needs, as the single unit can be positioned on the appropriate side of the bed. Users who rent or move frequently may prefer Bedjet's portable design that doesn't require commitment to a specific mattress system. Those who want some programmable temperature control but don't need comprehensive sleep tracking will find Bedjet's biorhythm technology adequate for their requirements. The compact size also makes it suitable for smaller bedrooms where space optimization is important.

### Final Considerations

When choosing between Eight Sleep and Bedjet, consumers should consider their primary sleep concerns, budget, and technological preferences. Eight Sleep represents a holistic sleep health investment with premium features and capabilities, while Bedjet offers focused thermal comfort at a more accessible price point. For users with chronic sleep issues related to temperature dysregulation or health conditions affecting sleep, Eight Sleep's comprehensive approach may justify the higher cost. For those seeking relief from occasional night sweats or seasonal temperature changes without extensive data tracking, Bedjet provides effective functionality at a fraction of the price.

Both systems have demonstrated reliability and user satisfaction in their respective niches. Eight Sleep appeals to tech enthusiasts and health optimizers who value data and precision, while Bedjet attracts practical consumers who want simple, effective temperature control without complexity. Ultimately, the choice depends on whether the user prioritizes comprehensive sleep optimization with health tracking (Eight Sleep) or straightforward thermal comfort with ease of use and affordability (Bedjet).

### Sources

[1] Eight Sleep Pod 3 Review: A Pricey but Game-Changing Sleep Setup: https://www.popularmechanics.com/technology/a60165942/eight-sleep-review/  
[2] BedJet 3 Review: Effective Cooling And Warming For Your Bed: https://www.youtube.com/watch?v=t6i1j3tw8mQ  
[3] Eight Sleep Pod 4 Cover: https://www.cnet.com/health/sleep/we-tested-the-eight-sleep-pod-4-cover/  
[4] BedJet vs Eight Sleep: https://www.skool.com/review-7926/bedjet-vs-eight-sleep  
[5] BedJet vs Eight Sleep: Which Bed Cooling System is Right for You?: https://www.rd.com/article/bedjet-vs-eight-sleep/  
[6] BedJet 3 Review: Cooling And Heating Done Well: https://www.nosleeplessnights.com/bedjet-3/  
[7] BedJet 3 Bed Climate Control Comfort System: https://bedjet.com/products/bedjet-3-climate-comfort-system-with-biorhythm-sleep-technology  
[8] Battle of the Smart Beds: Eight Sleep vs Sleep Number vs BedJet: https://www.soamjena.com/blog/smart-beds-cooling-tech